# 使用transformer的encoder和decoder替换baseline的encoder和decoder

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
import random
from torch.utils.data import Subset, DataLoader
import time



/home/xc/miniconda3/envs/nlp_datawhale/lib/python3.12/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


## 在训练的时候，把专业术语加入到词汇表中。在生成的时候，把生成的句子中的专业术语进行单独的替换。

In [15]:
# 定义数据集类
# 修改TranslationDataset类以处理术语
class TranslationDataset(Dataset):
    def __init__(self, filename, terminology):
        # train data: [(en, zh)]
        self.data = []
        with open(filename, 'r', encoding='utf-8') as f:
            for line in f:
                en, zh = line.strip().split('\t')
                self.data.append((en, zh))
        self.terminology = terminology
        
        # 创建词汇表，注意这里需要确保术语词典中的词也被包含在词汇表中
        self.en_tokenizer = get_tokenizer('basic_english')
        self.zh_tokenizer = list  # 使用字符级分词，list(str)直接将str按字符划分为一个list
        
        # count: 统计list\str的元素的个数，返回类似{'apple': 3, 'banana': 2, 'orange': 1}
        en_vocab = Counter(self.terminology.keys())  # 确保术语在词汇表中
        zh_vocab = Counter()
        for en, zh in self.data:
            en_vocab.update(self.en_tokenizer(en))
            zh_vocab.update(self.zh_tokenizer(zh))
        
        # 添加术语到词汇表。
        # Counter.most_common(n)：返回出现次数大于等于n的所有元素。例如上述n=2时，返回[('apple': 3), ('banana': 2)]
        # <pad> : padding; <sos> : begin; <eos> : end.
        self.en_vocab = ['<pad>', '<sos>', '<eos>'] + list(self.terminology.keys()) + [word for word, _ in en_vocab.most_common(10000)]
        self.zh_vocab = ['<pad>', '<sos>', '<eos>'] + [word for word, _ in zh_vocab.most_common(10000)]
        
        # {word: idx}
        self.en_word2idx = {word: idx for idx, word in enumerate(self.en_vocab)}
        self.zh_word2idx = {word: idx for idx, word in enumerate(self.zh_vocab)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        en, zh = self.data[idx]
        en_tensor = torch.tensor([self.en_word2idx.get(word, self.en_word2idx['<sos>']) for word in self.en_tokenizer(en)] + [self.en_word2idx['<eos>']])
        zh_tensor = torch.tensor([self.zh_word2idx.get(word, self.zh_word2idx['<sos>']) for word in self.zh_tokenizer(zh)] + [self.zh_word2idx['<eos>']])
        return en_tensor, zh_tensor

# 作用: padding
def collate_fn(batch):
    en_batch, zh_batch = [], []
    for en_item, zh_item in batch:
        en_batch.append(en_item)
        zh_batch.append(zh_item)
    
    # 对英文和中文序列分别进行填充
    en_batch = nn.utils.rnn.pad_sequence(en_batch, padding_value=0, batch_first=True)
    zh_batch = nn.utils.rnn.pad_sequence(zh_batch, padding_value=0, batch_first=True)
    
    return en_batch, zh_batch


In [ ]:
# str1 = [[1,2], [1,2,3], [2]]
# str1_pad = nn.utils.rnn.pad_sequence(str1, padding_value=0, batch_first=True)
# print(str1)

In [16]:
"""
使用transformer的encoder和decoder替换baseline的encoder和decoder
encoder : embed->dropoout->GRU
decoder : embed->dropoout->GRU->Linear
"""

class Encoder(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, d_model)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src shape: [batch_size, src_len]
        embedded = self.dropout(self.embedding(src))
        # embedded shape: [batch_size, src_len, d_model]
        outputs = self.transformer_encoder(embedded)
        # outputs shape: [batch_size, src_len, d_model]
        return outputs

class Decoder(nn.Module):
    def __init__(self, output_dim, d_model, num_heads, num_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.embedding = nn.Embedding(output_dim, d_model)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=num_heads)
        self.transformer_decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, z):
        # input shape: [batch_size, 1]
        # z shape: [n_layers, batch_size, d_model]
        
        embedded = self.dropout(self.embedding(input))
        # embedded shape: [batch_size, 1, d_model]
        
        output = self.transformer_decoder(embedded, z)
        # output shape: [batch_size, 1, d_model]
        
        prediction = self.fc_out(output.squeeze(1))
        # prediction shape: [batch_size, output_dim]
        
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src shape: [batch_size, src_len]
        # trg shape: [batch_size, trg_len]
        
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        _, hidden = self.encoder(src)
        
        # 输入decoder的<start token>。每次输入的都是一样的
        # input : [bs, 1]
        input = trg[:, 0].unsqueeze(1)  # Start token
        
        # 不停迭代decoder，生成新的token，直到满足预设长度
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden)
            outputs[:, t, :] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) # torch.Tensor.argmax(dim=1)：返回最大的alue的indices。这里返回[bs]
            # input需要变为[bs, 1]，所以需要unsqueeze(1)
            input = trg[:, t].unsqueeze(1) if teacher_force else top1.unsqueeze(1)

        return outputs

In [17]:
# 新增术语词典加载部分，{en, zh}
def load_terminology_dictionary(dict_file):
    terminology = {}
    with open(dict_file, 'r', encoding='utf-8') as f:
        for line in f:
            en_term, ch_term = line.strip().split('\t')
            terminology[en_term] = ch_term
    return terminology

In [18]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    for i, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[:, 1:].contiguous().view(-1, output_dim)
        trg = trg[:, 1:].contiguous().view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [19]:
# 主函数
if __name__ == '__main__':
    start_time = time.time()  # 开始计时

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    terminology = load_terminology_dictionary('../dataset/en-zh.dic') # dict

    # 加载数据
    dataset = TranslationDataset('../dataset/train.txt',terminology = terminology)
    # 选择数据集的前N个样本进行训练
    N = 1  #int(len(dataset) * 1)  # 或者你可以设置为数据集大小的一定比例，如 int(len(dataset) * 0.1)
    subset_indices = list(range(N))
    subset_dataset = Subset(dataset, subset_indices)
    train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

    # 定义模型参数
    INPUT_DIM = len(dataset.en_vocab)
    OUTPUT_DIM = len(dataset.zh_vocab)
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    HID_DIM = 512
    N_LAYERS = 2
    ENC_DROPOUT = 0.5
    DEC_DROPOUT = 0.5

    # 初始化模型
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
    model = Seq2Seq(enc, dec, device).to(device)

    # 定义优化器和损失函数
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=dataset.zh_word2idx['<pad>'])

    # 训练模型
    N_EPOCHS = 1
    CLIP = 1

    for epoch in range(N_EPOCHS):
        train_loss = train(model, train_loader, optimizer, criterion, CLIP)
        print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f}')
        
    # 在训练循环结束后保存模型
    torch.save(model.state_dict(), './translation_model_GRU.pth')
    
    end_time = time.time()  # 结束计时

    # 计算并打印运行时间
    elapsed_time_minute = (end_time - start_time)/60
    print(f"Total running time: {elapsed_time_minute:.2f} minutes")

Epoch: 01 | Train Loss: 8.453
Total running time: 0.08 minutes


### 在开发集上进行模型评价

In [20]:
import torch
from sacrebleu.metrics import BLEU
from typing import List

In [21]:
# 假设我们已经定义了TranslationDataset, Encoder, Decoder, Seq2Seq类

def load_sentences(file_path: str) -> List[str]:
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]

# 更新translate_sentence函数以考虑术语词典
def translate_sentence(sentence: str, model: Seq2Seq, dataset: TranslationDataset, terminology, device: torch.device, max_length: int = 50):
    model.eval()
    tokens = dataset.en_tokenizer(sentence) # token
    # 转化为idx
    tensor = torch.LongTensor([dataset.en_word2idx.get(token, dataset.en_word2idx['<sos>']) for token in tokens]).unsqueeze(0).to(device)  # [1, seq_len]
    
    with torch.no_grad():
        _, hidden = model.encoder(tensor)

    translated_tokens = []
    # 作为最初始的token
    input_token = torch.LongTensor([[dataset.zh_word2idx['<sos>']]]).to(device)  # [1, 1]

    for _ in range(max_length):
        output, hidden = model.decoder(input_token, hidden)
        top_token = output.argmax(1)
        translated_token = dataset.zh_vocab[top_token.item()]
        
        if translated_token == '<eos>':
            break
        
        # 如果翻译的词在术语词典中，则使用术语词典中的词
        if translated_token in terminology.values():
            for en_term, ch_term in terminology.items():
                if translated_token == ch_term:
                    translated_token = en_term
                    break
        
        translated_tokens.append(translated_token)
        input_token = top_token.unsqueeze(1)  # [1, 1]

    return ''.join(translated_tokens)


In [22]:
def evaluate_bleu(model: Seq2Seq, dataset: TranslationDataset, src_file: str, ref_file: str, terminology,device: torch.device):
    model.eval()
    src_sentences = load_sentences(src_file)
    ref_sentences = load_sentences(ref_file)
    
    translated_sentences = []
    for src in src_sentences:
        translated = translate_sentence(src, model, dataset, terminology, device)
        translated_sentences.append(translated)
    
    # 调用BLEU()，的出分数
    bleu = BLEU()
    score = bleu.corpus_score(translated_sentences, [ref_sentences])
    
    return score

In [25]:
# 主函数
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加载术语词典
    terminology = load_terminology_dictionary('../dataset/en-zh.dic')
    
    # 创建数据集实例时传递术语词典
    dataset = TranslationDataset('../dataset/train.txt', terminology)
    

    # 定义模型参数
    INPUT_DIM = len(dataset.en_vocab)
    OUTPUT_DIM = len(dataset.zh_vocab)
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    HID_DIM = 512
    N_LAYERS = 2
    ENC_DROPOUT = 0.5
    DEC_DROPOUT = 0.5

    # 初始化模型
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
    model = Seq2Seq(enc, dec, device).to(device)

    # 加载训练好的模型
    model.load_state_dict(torch.load('./translation_model_GRU.pth'))

    # 评估BLEU分数
    bleu_score = evaluate_bleu(model, dataset, '../dataset/dev_en.txt', '../dataset/dev_zh.txt', terminology = terminology,device = device)
    print(f'BLEU-4 score: {bleu_score.score:.2f}')

BLEU-4 score: 0.03


### 在测试集上进行推理

In [27]:
def inference(model: Seq2Seq, dataset: TranslationDataset, src_file: str, save_dir:str, terminology, device: torch.device):
    model.eval()
    src_sentences = load_sentences(src_file)
    
    translated_sentences = []
    for src in src_sentences:
        translated = translate_sentence(src, model, dataset, terminology, device)
        #print(translated)
        translated_sentences.append(translated)
        print(translated_sentences)

    # 将列表元素连接成一个字符串，每个元素后换行
    text = '\n'.join(translated_sentences)

    # 打开一个文件，如果不存在则创建，'w'表示写模式
    with open(save_dir, 'w', encoding='utf-8') as f:
        # 将字符串写入文件
        f.write(text)

    #return translated_sentences

In [28]:
# 主函数
if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # 加载术语词典
    terminology = load_terminology_dictionary('../dataset/en-zh.dic')
    # 加载数据集和模型
    dataset = TranslationDataset('../dataset/train.txt',terminology = terminology)

    # 定义模型参数
    INPUT_DIM = len(dataset.en_vocab)
    OUTPUT_DIM = len(dataset.zh_vocab)
    ENC_EMB_DIM = 256
    DEC_EMB_DIM = 256
    HID_DIM = 512
    N_LAYERS = 2
    ENC_DROPOUT = 0.5
    DEC_DROPOUT = 0.5

    # 初始化模型
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
    model = Seq2Seq(enc, dec, device).to(device)

    # 加载训练好的模型
    model.load_state_dict(torch.load('./translation_model_GRU.pth'))
    

    translated_sentences = inference(model, dataset, src_file="../dataset/test_en.txt", save_dir = f'../dataset/submit.txt', terminology = terminology, device = device)
    #print(translated_sentences)

['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫']
['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫', 'h柯話余剃峭峭杖唐葉呕呕叠庸蛟于眷歉柯柯媞康慰瘀崇迈员瘀崇迈的葉的的折的的折椭人人滴社社媞瘀适适适康']
['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫', 'h柯話余剃峭峭杖唐葉呕呕叠庸蛟于眷歉柯柯媞康慰瘀崇迈员瘀崇迈的葉的的折的的折椭人人滴社社媞瘀适适适康', '您楣螳全爛臊阿棺亡孰士岖魯魯·绯搪褶数扑牌挣悉夺夺您您恣í鹬搅搅崩崩迈窦窦扎欧褛禀藓桔擂５恣劲跤童鳖']
['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫', 'h柯話余剃峭峭杖唐葉呕呕叠庸蛟于眷歉柯柯媞康慰瘀崇迈员瘀崇迈的葉的的折的的折椭人人滴社社媞瘀适适适康', '您楣螳全爛臊阿棺亡孰士岖魯魯·绯搪褶数扑牌挣悉夺夺您您恣í鹬搅搅崩崩迈窦窦扎欧褛禀藓桔擂５恣劲跤童鳖', '冰楣亏幹幹歉柯媞媞大大喹嫌嫌涉嫌涉５蜢岁終溪代代代斐诘饿歧林姊o沿典谦谦谦棉嶙螂岭袋闻许许眺眺摇摇很']
['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫', 'h柯話余剃峭峭杖唐葉呕呕叠庸蛟于眷歉柯柯媞康慰瘀崇迈员瘀崇迈的葉的的折的的折椭人人滴社社媞瘀适适适康', '您楣螳全爛臊阿棺亡孰士岖魯魯·绯搪褶数扑牌挣悉夺夺您您恣í鹬搅搅崩崩迈窦窦扎欧褛禀藓桔擂５恣劲跤童鳖', '冰楣亏幹幹歉柯媞媞大大喹嫌嫌涉嫌涉５蜢岁終溪代代代斐诘饿歧林姊o沿典谦谦谦棉嶙螂岭袋闻许许眺眺摇摇很', '您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫']
['您楣螳全爛臊旋治悉悉折Σ拼拼酌將粉淫淫喝研膏锋币鲷鲷选的的的折的的几毒砜砜社杰辽汲迈楣葉葉读扑读柯枫', 'h柯話余剃峭峭杖唐葉呕呕叠庸蛟于眷歉柯柯媞康慰瘀崇迈员瘀崇迈的葉的的折的的折椭人人滴社社媞瘀适适适康', '您楣螳全爛臊阿棺亡孰士岖魯魯·绯搪褶数扑牌挣悉夺夺您您恣í鹬搅搅崩崩迈窦窦扎欧褛禀藓桔擂５恣劲跤童鳖', '冰楣亏幹幹歉柯媞媞大大喹嫌嫌涉嫌涉５蜢岁終

KeyboardInterrupt: 